In [ ]:
pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 25.7 MB/s eta 0:00:00


In [ ]:
pip install pytorch-forecasting torch lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import yfinance as yf  # for fetching stock data
import matplotlib.pyplot as plt
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss, MAE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
l1 = []
for i in os.listdir('/content/drive/MyDrive/processed'):
  l1.append(i.strip("processed_news_sentiment_").strip(".csv"))

In [ ]:
with open('/content/drive/MyDrive/all_tickers.txt', 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]
print(lines)

['ADBE', 'BX', 'MET', 'DLR', 'TJX', 'TT', 'BN', 'MA', 'CPRT', 'COST', 'NOC', 'UNP', 'BKNG', 'MDLZ', 'ADI', 'WM', 'TDG', 'CI', 'SYK', 'MU', 'ABBV', 'ABNB', 'PEP', 'NOW', 'LLY', 'ISRG', 'CSCO', 'EQIX', 'HCA', 'GILD', 'VRTX', 'APH', 'VZ', 'CL', 'MSI', 'FCX', 'CDNS', 'PSX', 'BAC', 'UAL', 'CHTR', 'MDT', 'BK', 'MS', 'ROP', 'CTAS', 'MSFT', 'ZTS', 'FDX', 'RGNX', 'AMAT', 'AON', 'AZO', 'AXP', 'UBER', 'CME', 'PH', 'AMP', 'CVX', 'XOM', 'ICE', 'PCG', 'NXPI', 'KLAC', 'NVDA', 'ACN', 'SCHW', 'SQ', 'CMG', 'SHW', 'SPG', 'AMD', 'O', 'URI', 'RTX', 'PNC', 'SBUX', 'ORCL', 'TXN', 'MMM', 'TMO', 'INTU', 'DHI', 'INTC', 'AMZN', 'MNST', 'UPS', 'NEE', 'ELV', 'EMR', 'ECL', 'WELL', 'AMT', 'GE', 'PCAR', 'MCD', 'PM', 'ADSK', 'CAT', 'GS', 'GD', 'FTNT', 'PYPL', 'BDX', 'KMI', 'FICO', 'D', 'DE', 'SRE', 'TSLA', 'RCL', 'ANET', 'ABT', 'CVS', 'META', 'SO', 'COF', 'PAYX', 'COP', 'DHR', 'MCK', 'CRWD', 'HD', 'ORLY', 'WFC', 'NEM', 'FANG', 'MCO', 'DIS', 'PG', 'ALL', 'T', 'C', 'ETN', 'GWW', 'NRG', 'NSC', 'SPGI', 'NKE', 'V', 'KO', '

In [ ]:
len(lines)

200

In [ ]:
def convert_data_type(df, time_cols=[], float_cols=[], cat_cols=[]):
    for col in time_cols:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
    for col in float_cols:
        df[col] = df[col].astype(float)
    for col in cat_cols:
        df[col] = df[col].astype('category')
    return df

# Define the stock tickers and the date range
tickers = lines
start_date = "2024-02-03"
end_date = "2024-11-03"

In [ ]:
# Download historical data from yfinance
df = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

[*********************100%***********************]  200 of 200 completed


In [ ]:
df.head()

Ticker                           ISRG                                      \
Price                            Open        High         Low       Close   
Date                                                                        
2024-02-05 00:00:00+00:00  384.470001  388.619995  380.570007  383.940002   
2024-02-06 00:00:00+00:00  385.260010  389.790009  384.119995  389.690002   
2024-02-07 00:00:00+00:00  390.380005  392.000000  385.720001  391.380005   
2024-02-08 00:00:00+00:00  389.510010  391.600006  385.359985  386.940002   
2024-02-09 00:00:00+00:00  387.459991  389.920013  383.049988  388.220001   

Ticker                                                  O             \
Price                       Adj Close   Volume       Open       High   
Date                                                                   
2024-02-05 00:00:00+00:00  383.940002  1575100  53.540001  53.610001   
2024-02-06 00:00:00+00:00  389.690002  1694300  52.750000  53.930000   
2024-02-07 00:00:00+00:00  391.380005  1458800  53.750000  53.889999   
2024-02-08 00:00:00+00:00  386.940002   998700  53.270000  53.520000   
2024-02-09 00:00:00+00:00  388.220001  1233900  53.029999  53.259998   

Ticker                                           ...         SYK              \
Price                            Low      Close  ...         Low       Close   
Date                                             ...                           
2024-02-05 00:00:00+00:00  52.610001  52.650002  ...  340.679993  342.910004   
2024-02-06 00:00:00+00:00  52.599998  53.750000  ...  338.899994  339.769989   
2024-02-07 00:00:00+00:00  53.290001  53.360001  ...  338.230011  342.720001   
2024-02-08 00:00:00+00:00  53.040001  53.160000  ...  337.079987  339.029999   
2024-02-09 00:00:00+00:00  52.270000  52.759998  ...  337.630005  341.959991   

Ticker                                                  TT              \
Price                       Adj Close   Volume        Open        High   
Date                                                                     
2024-02-05 00:00:00+00:00  340.565430  1539600  273.799988  275.429993   
2024-02-06 00:00:00+00:00  337.446869  1314400  272.029999  272.950012   
2024-02-07 00:00:00+00:00  340.376709   823900  272.890015  276.329987   
2024-02-08 00:00:00+00:00  336.711945  1445700  274.459991  274.459991   
2024-02-09 00:00:00+00:00  339.621918  1402300  269.899994  275.489990   

Ticker                                                                  
Price                             Low       Close   Adj Close   Volume  
Date                                                                    
2024-02-05 00:00:00+00:00  269.769989  271.799988  269.618164  1195400  
2024-02-06 00:00:00+00:00  268.600006  270.230011  268.060791  1144000  
2024-02-07 00:00:00+00:00  271.690002  273.429993  271.235077  1395300  
2024-02-08 00:00:00+00:00  269.609985  269.940002  267.773102  1678000  
2024-02-09 00:00:00+00:00  269.899994  275.440002  273.228943   959500  

[5 rows x 1200 columns]

In [ ]:
df = df.stack(level=0).reset_index()
df.columns = ['time', 'ticker', 'open', 'high', 'low', 'close', 'adjclose', 'volume']
df = convert_data_type(df, time_cols=['time'], float_cols=['open', 'high', 'low', 'close', 'volume'], cat_cols=['ticker'])

# Set the time as index and sort values by time
df = df.set_index('time', drop=True).reset_index()
df = df.groupby('ticker').apply(lambda x: x.sort_values('time')).reset_index(drop=True)


<ipython-input-15-48021d768e11>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=0).reset_index()
<ipython-input-15-48021d768e11>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby('ticker').apply(lambda x: x.sort_values('time')).reset_index(drop=True)
<ipython-input-15-48021d768e11>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly se

In [ ]:
df

,time,ticker,open,high,low,close,adjclose,volume
0,2024-02-05 00:00:00+00:00,AAPL,188.149994,189.250000,185.839996,187.679993,186.765015,69668800.0
1,2024-02-06 00:00:00+00:00,AAPL,186.860001,189.309998,186.770004,189.300003,188.377136,43490800.0
2,2024-02-07 00:00:00+00:00,AAPL,190.639999,191.050003,188.610001,189.410004,188.486588,53439000.0
3,2024-02-08 00:00:00+00:00,AAPL,189.389999,189.539993,187.350006,188.320007,187.401917,40962000.0
4,2024-02-09 00:00:00+00:00,AAPL,188.649994,189.990005,188.000000,188.850006,188.169144,45155200.0
...,...,...,...,...,...,...,...,...
37757,2024-10-28 00:00:00+00:00,ZTS,181.529999,183.100006,180.699997,182.759995,182.327942,1909700.0
37758,2024-10-29 00:00:00+00:00,ZTS,182.009995,184.410004,181.039993,181.270004,180.841476,1696000.0
37759,2024-10-30 00:00:00+00:00,ZTS,179.919998,183.809998,179.800003,182.740005,182.307999,1957800.0
37760,2024-10-31 00:00:00+00:00,ZTS,180.220001,181.580002,178.740005,178.779999,178.779999,2120000.0


In [ ]:
required_df = df[['ticker','time','close']]

In [ ]:
all_rdf = []

for i in os.listdir('/content/drive/MyDrive/processed'):
    name = i.strip("processed_news_sentiment_").strip(".csv")
    rdf = pd.read_csv(f'/content/drive/MyDrive/processed/{i}')
    rdf['ticker'] = name
    all_rdf.append(rdf)

for rdf in all_rdf:
    rdf.rename(columns={'Date': 'time'}, inplace=True)
    rdf['time'] = pd.to_datetime(rdf['time']).dt.strftime('%Y-%m-%d 00:00:00+00:00')

combined_df = pd.concat(all_rdf, ignore_index=True)

<ipython-input-18-55606edb89ca>:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(all_rdf, ignore_index=True)


In [ ]:
combined_df['time'] = pd.to_datetime(combined_df['time'], utc=True)
combined_df

,time,Overall Sentiment Score,Summary,ticker
0,2024-10-14 00:00:00+00:00,0.192184,Procter & Gamble (PG) closed the most recent t...,PG
1,2024-10-15 00:00:00+00:00,0.274811,"Procter & Gamble, Clorox, Church & Dwight and ...",PG
2,2024-10-16 00:00:00+00:00,0.363833,"LOS ANGELES, Oct. 16, 2024 /PRNewswire/ -- Wil...",PG
3,2024-10-17 00:00:00+00:00,0.224119,In today's rapidly changing and highly competi...,PG
4,2024-10-18 00:00:00+00:00,0.165140,The Procter & Gamble Company PG issued its fis...,PG
...,...,...,...,...
36265,2024-08-22 00:00:00+00:00,0.197780,Finding stocks that pay you can be rewarding.,VFC
36266,2024-09-30 00:00:00+00:00,0.257611,Style Box ETF report for ...,VFC
36267,2024-10-28 00:00:00+00:00,0.117855,"VF Corp., Cadence and F5 are all up nicely in ...",VFC
36268,2024-10-29 00:00:00+00:00,0.184373,VF Corporation VFC shares are trading higher T...,VFC


In [ ]:
def get_sentiment_label(score):
    if score <= -0.35:
        return "Bearish"
    elif -0.35 < score <= -0.15:
        return "Somewhat-Bearish"
    elif -0.15 < score < 0.15:
        return "Neutral"
    elif 0.15 <= score < 0.35:
        return "Somewhat-Bullish"
    else:
        return "Bullish"

combined_df['sentiment_Label'] = combined_df['Overall Sentiment Score'].apply(get_sentiment_label)

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for sentiment_Label and Summary
combined_df['label_embedding'] = combined_df['sentiment_Label'].apply(lambda x: model.encode(x))
combined_df['summary_embedding'] = combined_df['Summary'].apply(lambda x: model.encode(x))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(df['time'].dtype)
print(combined_df['time'].dtype)

datetime64[ns, UTC]
datetime64[ns, UTC]


In [ ]:
merged_df = pd.merge(df, combined_df,
                     on=['time', 'ticker'],
                     how='inner')

In [ ]:
merged_df

,time,ticker,open,high,low,close,adjclose,volume,Overall Sentiment Score,Summary,sentiment_Label,label_embedding,summary_embedding
0,2024-10-17 00:00:00+00:00,ABBV,190.539993,191.619995,188.460007,188.570007,188.570007,4419000.0,0.200902,AbbVie (ABBV) reachead $188.57 at the closing ...,Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.11585849, -0.08680976, -0.03160621, -0.061..."
1,2024-10-18 00:00:00+00:00,ABBV,188.970001,189.690002,188.160004,188.860001,188.860001,5607900.0,0.223410,J&J tops third-quarter earnings and sales esti...,Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.072600074, -0.11379021, -0.026387982, -0.0..."
2,2024-10-22 00:00:00+00:00,ABBV,186.320007,189.100006,186.229996,188.759995,188.759995,4210400.0,0.085425,PFE's non-COVID drugs and potential contributi...,Neutral,"[-0.05107651, -0.013697355, -0.05514741, 0.000...","[-0.0074317656, -0.06831565, -0.051426172, -0...."
3,2024-10-23 00:00:00+00:00,ABBV,188.699997,189.270004,187.500000,187.880005,187.880005,4266700.0,0.180269,AbbVie (ABBV) doesn't possess the right combin...,Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.07981085, -0.106803425, -0.016921563, -0.0..."
4,2024-10-24 00:00:00+00:00,ABBV,188.429993,190.470001,188.169998,189.649994,189.649994,6741800.0,0.152407,"In the closing of the recent trading day, AbbV...",Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.09336783, -0.002650328, -0.03376093, -0.02..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16472,2024-10-25 00:00:00+00:00,ZTS,181.490005,182.029999,179.669998,180.009995,179.584442,2622900.0,0.352424,Finding stocks expected to beat quarterly earn...,Bullish,"[0.008234721, 0.029443769, -0.021321895, 0.059...","[-0.050197657, 0.005108781, 0.041490067, 0.044..."
16473,2024-10-28 00:00:00+00:00,ZTS,181.529999,183.100006,180.699997,182.759995,182.327942,1909700.0,0.199263,Zoetis (ZTS) closed at $182.76 in the latest t...,Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.044412978, 0.018267635, -0.019620217, 0.02..."
16474,2024-10-29 00:00:00+00:00,ZTS,182.009995,184.410004,181.039993,181.270004,180.841476,1696000.0,0.148615,Zevra Therapeutics (ZVRA) doesn't possess the ...,Neutral,"[-0.05107651, -0.013697355, -0.05514741, 0.000...","[-0.072585054, -0.032044634, -0.020095231, 0.0..."
16475,2024-10-30 00:00:00+00:00,ZTS,179.919998,183.809998,179.800003,182.740005,182.307999,1957800.0,0.124575,Besides Wall Street's top -and-bottom-line est...,Neutral,"[-0.05107651, -0.013697355, -0.05514741, 0.000...","[-0.061699428, -0.00273157, -0.0077714217, -0...."


In [ ]:
df = merged_df

In [ ]:
def split_dataset(df):
    df = df.sort_values(['ticker', 'time'])
    df['time_idx'] = df.groupby('ticker').cumcount()

    # Assuming 30 days per month and 7 days per week
    total_days = df.groupby('ticker').size().max()
    test_days = 30  # One week for test
    val_days = 30  # One month for validation
    train_days = total_days - val_days - test_days

    training_data = df[df['time_idx'] < train_days]
    validation_data = df[(df['time_idx'] >= train_days) & (df['time_idx'] < train_days + val_days)]
    test_data = df[df['time_idx'] >= train_days + val_days]

    return training_data, validation_data, test_data

# Apply the function to your DataFrame
training_df, validation_df, test_df = split_dataset(df)

In [ ]:
def verify_split(training_df, validation_df, test_df, expected_val_test_entries=30):
    train_counts = training_df.groupby('ticker').size()
    val_counts = validation_df.groupby('ticker').size()
    test_counts = test_df.groupby('ticker').size()

    print("Sample of training set ticker counts:")
    print(train_counts.head())
    print("\nSample of validation set ticker counts:")
    print(val_counts.head())
    print("\nSample of test set ticker counts:")
    print(test_counts.head())

    all_tickers = set(train_counts.index) | set(val_counts.index) | set(test_counts.index)

    validation_issues = []
    test_issues = []

    for ticker in all_tickers:
        if ticker in val_counts and val_counts[ticker] != expected_val_test_entries:
            validation_issues.append(f"Ticker {ticker}: {val_counts[ticker]} entries (expected {expected_val_test_entries})")
        if ticker in test_counts and test_counts[ticker] != expected_val_test_entries:
            test_issues.append(f"Ticker {ticker}: {test_counts[ticker]} entries (expected {expected_val_test_entries})")

    if validation_issues:
        print("\nValidation set issues:")
        for issue in validation_issues[:5]:  # Print first 5 issues
            print(issue)
        print(f"... and {len(validation_issues) - 5} more issues") if len(validation_issues) > 5 else None

    if test_issues:
        print("\nTest set issues:")
        for issue in test_issues[:5]:  # Print first 5 issues
            print(issue)
        print(f"... and {len(test_issues) - 5} more issues") if len(test_issues) > 5 else None

    if not validation_issues and not test_issues:
        print(f"\nVerification passed: All tickers have exactly {expected_val_test_entries} entries in both validation and test sets.")
    else:
        print(f"\nVerification failed: Some tickers do not have {expected_val_test_entries} entries in validation or test sets.")

    print(f"\nTotal tickers: {len(all_tickers)}")
    print(f"Tickers in training set: {len(train_counts)}")
    print(f"Tickers in validation set: {len(val_counts)}")
    print(f"Tickers in test set: {len(test_counts)}")

# Verify the split datasets
verify_split(training_df, validation_df, test_df)

Sample of training set ticker counts:
ticker
ABBV     11
ABNB     10
ABT     117
ACN     117
ADBE    117
dtype: int64

Sample of validation set ticker counts:
ticker
ABT     27
ACN     30
ADBE    30
ADI      4
ADSK    29
dtype: int64

Sample of test set ticker counts:
ticker
ACN      4
ADBE    22
AMAT    22
AMGN    24
BKNG    13
dtype: int64

Validation set issues:
Ticker ELV: 11 entries (expected 30)
Ticker AZO: 10 entries (expected 30)
Ticker ABT: 27 entries (expected 30)
Ticker NXPI: 9 entries (expected 30)
Ticker MET: 12 entries (expected 30)
... and 29 more issues

Test set issues:
Ticker ISRG: 17 entries (expected 30)
Ticker O: 18 entries (expected 30)
Ticker NEM: 29 entries (expected 30)
Ticker PGR: 2 entries (expected 30)
Ticker COP: 12 entries (expected 30)
... and 28 more issues

Verification failed: Some tickers do not have 30 entries in validation or test sets.

Total tickers: 189
Tickers in training set: 189
Tickers in validation set: 69
Tickers in test set: 35


In [ ]:
print(validation_df.shape)  # Check the shape of validation data
print(validation_df.head())  # Print the first few rows of validation data
print(validation_df['ticker'].value_counts())  # Check how many entries per ticker


(1591, 14)
                         time ticker        open        high         low  \
138 2024-09-09 00:00:00+00:00    ABT  114.459999  116.730003  114.320000   
139 2024-09-11 00:00:00+00:00    ABT  116.389999  116.720001  114.720001   
140 2024-09-12 00:00:00+00:00    ABT  116.500000  117.309998  115.589996   
141 2024-09-13 00:00:00+00:00    ABT  116.019997  117.050003  115.940002   
142 2024-09-16 00:00:00+00:00    ABT  117.000000  117.680000  116.419998   

          close    adjclose     volume  Overall Sentiment Score  \
138  116.430000  115.883850  5101000.0                 0.246889   
139  116.599998  116.053047  6535600.0                 0.213804   
140  116.830002  116.281975  4349300.0                 0.300135   
141  116.400002  115.853989  3350100.0                 0.390293   
142  117.629997  117.078217  3637100.0                 0.289804   

                                               Summary   sentiment_Label  \
138  This dynamic big pharma is undergoing a few ke..

In [ ]:
training_df.head()

,time,ticker,open,high,low,close,adjclose,volume,Overall Sentiment Score,Summary,sentiment_Label,label_embedding,summary_embedding,time_idx
0,2024-10-17 00:00:00+00:00,ABBV,190.539993,191.619995,188.460007,188.570007,188.570007,4419000.0,0.200902,AbbVie (ABBV) reachead $188.57 at the closing ...,Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.11585849, -0.08680976, -0.03160621, -0.061...",0
1,2024-10-18 00:00:00+00:00,ABBV,188.970001,189.690002,188.160004,188.860001,188.860001,5607900.0,0.223410,J&J tops third-quarter earnings and sales esti...,Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.072600074, -0.11379021, -0.026387982, -0.0...",1
2,2024-10-22 00:00:00+00:00,ABBV,186.320007,189.100006,186.229996,188.759995,188.759995,4210400.0,0.085425,PFE's non-COVID drugs and potential contributi...,Neutral,"[-0.05107651, -0.013697355, -0.05514741, 0.000...","[-0.0074317656, -0.06831565, -0.051426172, -0....",2
3,2024-10-23 00:00:00+00:00,ABBV,188.699997,189.270004,187.500000,187.880005,187.880005,4266700.0,0.180269,AbbVie (ABBV) doesn't possess the right combin...,Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.07981085, -0.106803425, -0.016921563, -0.0...",3
4,2024-10-24 00:00:00+00:00,ABBV,188.429993,190.470001,188.169998,189.649994,189.649994,6741800.0,0.152407,"In the closing of the recent trading day, AbbV...",Somewhat-Bullish,"[0.044144712, 0.00897564, 0.0045142593, 0.0171...","[-0.09336783, -0.002650328, -0.03376093, -0.02...",4


In [ ]:
# For training_df
training_df['label_embedding'] = training_df['label_embedding'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
training_df['summary_embedding'] = training_df['summary_embedding'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)

# For validation_df
validation_df['label_embedding'] = validation_df['label_embedding'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
validation_df['summary_embedding'] = validation_df['summary_embedding'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)

# For test_df
test_df['label_embedding'] = test_df['label_embedding'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
test_df['summary_embedding'] = test_df['summary_embedding'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)

In [ ]:
max_prediction_length = 6  # One week
max_encoder_length = 5  # One month, adjust if needed

training_dataset = TimeSeriesDataSet(
    training_df,
    time_idx="time_idx",
    target="close",
    group_ids=["ticker"],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["ticker"],
    static_reals=["label_embedding", "summary_embedding"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["close"],  # Includes sentiment score
    target_normalizer=GroupNormalizer(groups=["ticker"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)


# Create validation and test datasets
validation_dataset = TimeSeriesDataSet.from_dataset(training_dataset, validation_df, stop_randomization=True)
test_dataset = TimeSeriesDataSet.from_dataset(training_dataset, test_df, stop_randomization=True)

/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/data/timeseries.py:1301: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 2 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__ticker': 'MCO'}, {'__group_id__ticker': 'SQ'}]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/data/timeseries.py:1301: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__ticker': 'AXP'}]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/data/timeseries.py:1301: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction l

In [ ]:
batch_size = 128  # Increased batch size for faster training
train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)
test_dataloader = test_dataset.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)

In [ ]:
pl.seed_everything(42)

tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=0.001,  # Reduced learning rate for better stability
    hidden_size=64,  # Increased hidden size for more capacity
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=32,
    loss=QuantileLoss(),
    optimizer="adamw",  # Changed to AdamW optimizer
    reduce_on_plateau_patience=4,
)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")

In [ ]:
import torch

print("Is CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

Is CUDA available: True
Current device: Tesla T4


In [ ]:
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",  # Colab will automatically detect and use GPU if available
    devices=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    log_every_n_steps=50,
    val_check_interval=None,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader)


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 5.7 K  | train
3  | prescalers                         | ModuleDict                      | 512    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 35.1 K | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 20.7 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 13.7 K | train
7  | static_context_variable_selectio

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# Print the first batch to see its structure
for batch in test_dataloader:
    print("Batch structure:", type(batch), "Length:", len(batch))
    for item in batch:
        print("Item type:", type(item), "Shape:", item.shape if hasattr(item, 'shape') else "No shape")
    break  # Just print the first batch

Batch structure: <class 'tuple'> Length: 2
Item type: <class 'dict'> Shape: No shape
Item type: <class 'tuple'> Shape: No shape


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
from pytorch_forecasting.metrics import MAE

# Perform prediction
predictions = tft.predict(test_dataloader, return_x=True)

# Determine the device
device = next(tft.parameters()).device

# Move predictions to CPU (since we'll convert to numpy later)
predicted_values = predictions.output.cpu().numpy()

# Calculate MAE for the test set
actuals = torch.cat([y[0].cpu() for x, y in iter(test_dataloader)]).numpy() # Move actuals to CPU as well

predicted_values = predicted_values.ravel()
actuals = actuals.ravel()

# Check for NaN values
nan_mask = np.isnan(predicted_values) | np.isnan(actuals)

if np.any(nan_mask):
    print("NaN values found:")
    print(f"Total NaN values: {np.sum(nan_mask)}")

    # Remove NaN values
    valid_indices = ~nan_mask
    predicted_values = predicted_values[valid_indices]
    actuals = actuals[valid_indices]

# Calculate MAE
mae_value = mean_absolute_error(actuals, predicted_values)

# Calculate MSE
mse = mean_squared_error(actuals, predicted_values)

print(f"Test MAE: {mae_value}")
print(f"Test MSE: {mse}")

# Additional information
print(f"Number of samples after removing NaNs: {len(predicted_values)}")
print(f"Predictions range: {np.min(predicted_values)} to {np.max(predicted_values)}")
print(f"Actuals range: {np.min(actuals)} to {np.max(actuals)}")

# Diagnostic information
print("\nDiagnostic Information:")
print(f"Shape of predicted_values: {predicted_values.shape}")
print(f"Shape of actuals: {actuals.shape}")
print(f"Type of predicted_values: {type(predicted_values)}")
print(f"Type of actuals: {type(actuals)}")



INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


NaN values found:
Total NaN values: 914
Test MAE: 32.756370544433594
Test MSE: 14941.6591796875
Number of samples after removing NaNs: 2722
Predictions range: 19.56686019897461 to 4103.7080078125
Actuals range: 21.260000228881836 to 4749.10986328125

Diagnostic Information:
Shape of predicted_values: (2722,)
Shape of actuals: (2722,)
Type of predicted_values: <class 'numpy.ndarray'>
Type of actuals: <class 'numpy.ndarray'>


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import numpy as np
from pytorch_forecasting.metrics import MAE

# Perform prediction
predictions = tft.predict(test_dataloader, return_x=True)

# Determine the device
device = next(tft.parameters()).device

# Move predictions to CPU (since we'll convert to numpy later)
predicted_values = predictions.output.cpu().numpy()

# Calculate actual values for the test set
actuals = torch.cat([y[0].cpu() for x, y in iter(test_dataloader)]).numpy()  # Move actuals to CPU as well

# Flatten predictions and actuals
predicted_values = predicted_values.ravel()
actuals = actuals.ravel()

# Check for NaN values
nan_mask = np.isnan(predicted_values) | np.isnan(actuals)

if np.any(nan_mask):
    print("NaN values found:")
    print(f"Total NaN values: {np.sum(nan_mask)}")

    # Remove NaN values
    valid_indices = ~nan_mask
    predicted_values = predicted_values[valid_indices]
    actuals = actuals[valid_indices]

# Slice for the first 4 points
predicted_4 = predicted_values[:4]
actuals_4 = actuals[:4]

# Slice for the first 14 points
predicted_14 = predicted_values[:14]
actuals_14 = actuals[:14]

# Calculate MAE and MSE for first 4 points
mae_4 = mean_absolute_error(actuals_4, predicted_4)
mse_4 = mean_squared_error(actuals_4, predicted_4)

# Calculate MAE and MSE for first 14 points
mae_14 = mean_absolute_error(actuals_14, predicted_14)
mse_14 = mean_squared_error(actuals_14, predicted_14)

# Output results
print("First 4 Points:")
print(f"MAE: {mae_4}")
print(f"MSE: {mse_4}")
print(f"Prediction Range: {predicted_4.min()} to {predicted_4.max()}")
print(f"Actual Range: {actuals_4.min()} to {actuals_4.max()}")
print("\nFirst 14 Points:")
print(f"MAE: {mae_14}")
print(f"MSE: {mse_14}")
print(f"Prediction Range: {predicted_14.min()} to {predicted_14.max()}")
print(f"Actual Range: {actuals_14.min()} to {actuals_14.max()}")

# Additional information
print("\nAdditional Diagnostic Information:")
print(f"Shape of predicted_values: {predicted_values.shape}")
print(f"Shape of actuals: {actuals.shape}")
print(f"Type of predicted_values: {type(predicted_values)}")
print(f"Type of actuals: {type(actuals)}")


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


NaN values found:
Total NaN values: 914
First 4 Points:
MAE: 9.676528930664062
MSE: 96.99581909179688
Prediction Range: 352.2535400390625 to 517.509521484375
Actual Range: 344.82000732421875 to 507.2200012207031

First 14 Points:
MAE: 20.594791412353516
MSE: 492.6856994628906
Prediction Range: 352.2535400390625 to 529.4994506835938
Actual Range: 344.82000732421875 to 507.2200012207031

Additional Diagnostic Information:
Shape of predicted_values: (2722,)
Shape of actuals: (2722,)
Type of predicted_values: <class 'numpy.ndarray'>
Type of actuals: <class 'numpy.ndarray'>
